In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import catboost

In [2]:
# Loading Data

train = pd.read_csv('data/train.csv')
songs = pd.read_csv('data/songs.csv')
song_labels = pd.read_csv('data/song_labels.csv')
test = pd.read_csv('data/test.csv')
save_for_later = pd.read_csv('data/save_for_later.csv')
dummy_submission = pd.read_csv('data/dummy_submission.csv')

In [3]:
customer_groups = train.groupby('customer_id')

In [7]:
customer_id_list = train['customer_id'].unique()

In [8]:
customer_groups.get_group('J15604')

,customer_id,song_id,score
3191,J15604,72,4
14799,J15604,321,3
14899,J15604,109,4
19831,J15604,6063,3
31449,J15604,484,5
...,...,...,...
694538,J15604,36,3
696375,J15604,2,4
698729,J15604,121,4
699770,J15604,641,5


In [9]:
song_labels_new = song_labels.groupby('label_id').sum().sort_values('count', ascending = False)
song_labels_new.reset_index(inplace = True)
principal_song_labels = song_labels_new['label_id'].to_numpy()[:3]

for i in principal_song_labels:
    label = song_labels[song_labels['label_id'] == i]
    label = label.reset_index().drop(['index', 'label_id'], axis = 1)
    label[f'count{i}'] = label['count']
    label.drop(['count'], axis = 1, inplace = True)
    songs = pd.merge(songs, label, on = 'platform_id', how = 'left')
    
for i in principal_song_labels:
    songs[f'count{i}'] = songs[f'count{i}'].fillna(0)
    
songs = songs.drop(['platform_id'], axis = 1)

In [10]:
songs

,song_id,released_year,language,number_of_comments,count30574,count8717,count11557
0,8328,2013.0,eng,1936,1759.0,215.0,295.0
1,8591,1925.0,eng,420,594.0,17.0,221.0
2,718,1929.0,eng,5485,281.0,0.0,1710.0
3,6593,1968.0,NaN,200,5174.0,186.0,45.0
4,743,2002.0,en-US,10411,30837.0,2903.0,2190.0
...,...,...,...,...,...,...,...
9976,2048,2013.0,eng,4793,1909.0,294.0,86.0
9977,2552,1952.0,eng,1365,1374.0,482.0,206.0
9978,6741,2006.0,en-US,1296,689.0,258.0,284.0
9979,4564,2012.0,eng,1961,2844.0,212.0,145.0


In [11]:
songs.drop_duplicates('song_id', keep = 'first', inplace = True)

,song_id,released_year,language,number_of_comments,count30574,count8717,count11557
0,8328,2013.0,eng,1936,1759.0,215.0,295.0
1,8591,1925.0,eng,420,594.0,17.0,221.0
2,718,1929.0,eng,5485,281.0,0.0,1710.0
3,6593,1968.0,NaN,200,5174.0,186.0,45.0
4,743,2002.0,en-US,10411,30837.0,2903.0,2190.0
...,...,...,...,...,...,...,...
9976,2048,2013.0,eng,4793,1909.0,294.0,86.0
9977,2552,1952.0,eng,1365,1374.0,482.0,206.0
9978,6741,2006.0,en-US,1296,689.0,258.0,284.0
9979,4564,2012.0,eng,1961,2844.0,212.0,145.0


In [13]:
group = customer_groups.get_group('J15604')

In [14]:
group

,customer_id,song_id,score
3191,J15604,72,4
14799,J15604,321,3
14899,J15604,109,4
19831,J15604,6063,3
31449,J15604,484,5
...,...,...,...
694538,J15604,36,3
696375,J15604,2,4
698729,J15604,121,4
699770,J15604,641,5


In [15]:
X_train = pd.merge(group, songs, on = ['song_id'], how = 'left')

In [16]:
Y_train = X_train['score']

In [17]:
X_train.drop(['score'], axis = 1, inplace = True)

In [18]:
X_train.drop('song_id', axis = 1, inplace = True)

In [19]:
X_train['released_year'] = X_train['released_year'].fillna(-999)
X_train['language'] = X_train['language'].fillna('none')
X_train['number_of_comments'] = X_train['number_of_comments'].fillna(-999)

In [20]:
from catboost import CatBoostRegressor

In [21]:
model = CatBoostRegressor()

In [23]:
import time
start = time.time()
model.fit(X_train, Y_train, cat_features = [0, 2], silent = True)
print(time.time()-start)

23.966368198394775


In [26]:
output.sort_index(axis = 0)

NameError: name 'output' is not defined